In [ ]:
import praw
import time
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import asyncio

# Reddit API credentials
client_id = 'YOUR_REDDIT_CLIENT_ID'
client_secret = 'YOUR_REDDIT_CLIENT_SECRET'
user_agent = 'YOUR_REDDIT_USER_AGENT'

# Initialize PRAW API
reddit = praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

# List of subreddits to fetch posts
healthcare_subreddits = [
    'healthcare', 'medicine', 'mentalhealth', 'healthcarepolicy', 'AffordableCareAct',
    'universalhealthcare', 'HealthPolicy', 'Medicare', 'Health', 'AskDocs',
    'healthcareworkers', 'MedicalEthics', 'medtech', 'PublicHealth'
]

economics_subreddits = [
    'economics', 'finance', 'worldnews', 'politics', 'investing', 'USPolitics',
    'libertarian', 'Business', 'Economics101', 'PersonalFinance', 'FinancialIndependence',
    'Economists', 'MacroEconomics', 'PoliticalEconomy'
]

crime_subreddits = [
    'TrueCrime', 'criminology', 'politics', 'law', 'USPolitics', 'CriminalJustice',
    'TrueCrimeDiscussion', 'CriminalJusticeReform', 'Law', 'justice',
    'PoliceReform', 'LegalAdvice', 'LegalTips', 'CriminalJustice', 'PrisonReform'
]

def clean_text(text):
    """Clean the text to remove unwanted characters."""
    return ' '.join(re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

def get_sentiment(text):
    """Return sentiment of the text as positive, neutral, or negative."""
    analysis = TextBlob(clean_text(text))
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity == 0:
        return 'neutral'
    else:
        return 'negative'

def check_rate_limit():
    """Check Reddit's rate limit status and sleep if needed."""
    while True:
        if reddit.auth.limits['remaining'] == 0:
            reset_time = reddit.auth.limits['reset']
            sleep_time = reset_time - time.time() + 10  # Adding some buffer
            print(f"Rate limit hit. Sleeping for {sleep_time} seconds.")
            time.sleep(sleep_time)
        else:
            break

def get_posts(subreddit_list, limit=100):
    """Fetch posts from a list of subreddits."""
    posts = []
    for subreddit in subreddit_list:
        check_rate_limit()  # Check rate limit before making a request
        print(f"Fetching posts from subreddit: {subreddit}")
        for submission in reddit.subreddit(subreddit).hot(limit=limit):
            posts.append({
                'title': submission.title,
                'text': submission.selftext,
                'sentiment': get_sentiment(submission.title + " " + submission.selftext)
            })
    return posts

def create_pie_chart(posts, title):
    """Create and display a pie chart of sentiment distribution."""
    if not posts:
        print(f"No posts found for {title}")
        return

    positive_posts = [post for post in posts if post['sentiment'] == 'positive']
    negative_posts = [post for post in posts if post['sentiment'] == 'negative']
    neutral_posts = [post for post in posts if post['sentiment'] == 'neutral']

    print(f"Positive posts for {title}: {100*len(positive_posts)/len(posts)} %")
    print(f"Negative posts for {title}: {100*len(negative_posts)/len(posts)} %")
    print(f"Neutral posts for {title}: {100*(len(posts) - len(negative_posts) - len(positive_posts))/len(posts)} %")

    # Pie chart representation
    positive_pie = 100*len(positive_posts)/len(posts)
    negative_pie = 100*len(negative_posts)/len(posts)
    neutral_pie = 100*(len(posts) - len(negative_posts) - len(positive_posts))/len(posts)

    sizes = [positive_pie, negative_pie, neutral_pie]
    labels = 'Positive Posts', 'Negative Posts', 'Neutral Posts'
    explode = (0, 0.1, 0.1)  # To highlight 'Negative' and 'Neutral'

    fig, ax = plt.subplots()
    ax.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)
    ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

    ax.set_title(f"Sentiment Distribution for {title}")  # Title for the chart
    plt.show()
    plt.savefig(f'{title}_pie_chart.jpeg')

def main():
    # Fetch and analyze Reddit posts related to Healthcare policies
    healthcare_posts = get_posts(healthcare_subreddits)
    create_pie_chart(healthcare_posts, "Healthcare Policies")

    # Fetch and analyze Reddit posts related to Economics policies
    economics_posts = get_posts(economics_subreddits)
    create_pie_chart(economics_posts, "Economics Policies")

    # Fetch and analyze Reddit posts related to Crime policies
    crime_posts = get_posts(crime_subreddits)
    create_pie_chart(crime_posts, "Crime Policies")

if __name__ == "__main__":
    main()